In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, projectUMAP, fitGMM, projectTSNE
from open_spiel.python.examples.ubc_sample_game_tree import NodeType

from auctions.webutils import *



import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Category10_10, Magma256, Spectral10, Category20_20
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

output_notebook()


Loading BokehJS ...

In [2]:
# OPTIONS
EXPERIMENT_NAME = 'mar24'
RUN_NAME = 'large_game_2-mar17lstm-100'
PROJECTION_METHOD = 'TSNE'
player = 0 

In [3]:

### Load checkpoint
checkpoint = get_checkpoint_by_name(EXPERIMENT_NAME, RUN_NAME)
game = load_game(checkpoint.game)
env_and_model = db_checkpoint_loader(checkpoint)

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing


RuntimeError: Error(s) in loading state_dict for AuctionRNN:
	size mismatch for rnn.weight_ih_l0: copying a param with shape torch.Size([512, 18]) from checkpoint, the shape in current model is torch.Size([512, 14]).

In [ ]:
### Optionally restrict to one type

env_and_model.env._chance_event_sampler = UBCChanceEventSampler(deterministic_types=[0, None])

In [ ]:
checkpoint

In [ ]:
### Sample 

trees = sample_game_tree(env_and_model, num_samples=1000, seed=1, include_embeddings=True)

In [ ]:
### Flatten trees and then restrict to a single player

df = flatten_trees(trees).query('embedding.notna()', engine='python')
df['index'] = df.index
df = df.reindex(df.index.repeat(df.num_plays))
dfp = df.query(f'player_id == {player}').copy()
dfp = dfp.drop([c for c in dfp.columns if c.startswith(f'avg_p{player}')], axis=1)



In [ ]:
# Project onto a 2D embedding
embeddings = np.stack(dfp['embedding'].values).squeeze()
if PROJECTION_METHOD == 'TSNE':
    proj = projectTSNE(embeddings, perplexity=30, early_exaggeration=100)
elif PROJECTION_METHOD == "PCA":
    proj, variance = projectPCA(embeddings)
elif PROJECTION_METHOD == "UMAP":
    proj = projectUMAP(embeddings, n_neighbors=200, min_dist=0.8)
else:
    raise ValueError(f"Unkown projection method {PROJECTION_METHOD}")
dfp[f'{PROJECTION_METHOD}_0'] = proj[:, 0]
dfp[f'{PROJECTION_METHOD}_1'] = proj[:, 1]


In [ ]:
### TODO: Note this is fixed to 20

gmm, clusters, scores = fitGMM(embeddings, verbose=True, trials=[20])
dfp['clusters'] = list(map(str, clusters))
dfp['clusters'] = dfp['clusters'].astype('category')

In [ ]:
### Now we sample again and track cluster information
cluster_fn = lambda x: gmm.predict(x)[0]
trees_with_clusters = sample_game_tree(env_and_model, num_samples=1000, seed=1, include_embeddings=True, clusterer=cluster_fn)

In [ ]:
df_with_clusters = flatten_trees(trees_with_clusters).query(f'player_id == {player}')

In [ ]:
matrix = df_with_clusters.query(f'type != {NodeType.ACTION}').groupby(['round', 'cluster_from', 'cluster'])['num_plays'].sum().unstack().fillna(0)
matrix.to_csv("data.csv")

In [ ]:
USEFUL_FIELDS = ['avg_p1_type_1', 'start_of_round_exposure']
df_with_clusters.query('cluster_from == 1').groupby('cluster')[USEFUL_FIELDS].describe()

In [ ]:
# df2.query('cluster_from == 1 and cluster == 10')[['avg_p1_type_1']].describe()

plt_x, plt_y = list(zip(*list(scores.items())))
plt.plot(plt_x, plt_y)
plt.xlabel('Number of clusters')
plt.ylabel('AIC score')

In [ ]:
### Plot clusters. Note that we remove duplicates here, though they will still feature heavily in the clustering algorithm

df_for_plotting = dfp.drop_duplicates(subset=['index']).copy()

plt.figure(figsize=(10, 10))
plt.scatter(
    x=df_for_plotting[f'{PROJECTION_METHOD}_0'],
    y=df_for_plotting[f'{PROJECTION_METHOD}_1'],
    c=df_for_plotting['clusters'].astype(int),
    cmap='tab20',
    s=50,
    alpha=0.5
)
plt.colorbar()
plt.show()

In [ ]:
# Plot all numeric columns
numerics = ['category', 'int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
newdf = dfp.select_dtypes(include=numerics)
IGNORE = ['type', 'depth', 'player_id', 'num_plays', 'index', 'pct_plays', 'pca_0', 'pca_1', 'umap_0', 'umap_1', 'TSNE_0', 'TSNE_1']
plots = []
for k in newdf.columns:
    if k not in IGNORE and f'avg_{k}' not in IGNORE:
        plot = plot_embedding(df_for_plotting, color_col=k, reduction_method=PROJECTION_METHOD, fast=False)
        plots.append(plot)



In [ ]:
# plots_to_html(plots, 'rounds.html')
# plots_to_html(plots, 'rounds_only2.html')

In [ ]:
for plot in plots:
    show(plot)


In [ ]:
# dfp.groupby('clusters')['round'].describe()

In [ ]:
# dfp.query('clusters == "13"').describe()

In [ ]:
df_with_clusters.columns

In [ ]:
from sklearn import tree
import graphviz
# Old code for using exact same features
# FEATURE_NAMES = ['round'] + [f'SoR profit {bundles[i]}' for i in range(len(bundles))] \
# + [f'Clock profit {bundles[i]}' for i in range(len(bundles))] + ['activity', 'SoR exposure'] \
# + [f'Price {num_to_letter(i)}' for i in range(num_products)] + [f'Holdings {num_to_letter(i)}' for i in range(num_products)] + [f'Agg {num_to_letter(i)}' for i in range(num_products)]
# X = df_with_clusters['feature_vector'].values
# X = np.array([np.array(x) for x in X])

# Restrict to interpretable features
FEATURE_COLS = ['round', 'activity', 'start_of_round_exposure'] + [c for c in df_with_clusters.columns if any((c.startswith(z) for z in ['Processed', 'Price Increments', 'Agg Demand']))]
X = df_with_clusters[FEATURE_COLS].values
Y = df_with_clusters['cluster']
clf = tree.DecisionTreeClassifier(min_impurity_decrease=0.01)
clf = clf.fit(X, Y)
# TODO: Seems very odd Price Increments features are never used....


In [ ]:
bundles = action_to_bundles(env_and_model.game_config['licenses'])
num_products = len(env_and_model.game_config['licenses'])

dot_data = tree.export_graphviz(clf, out_file=None, feature_names=FEATURE_COLS,  class_names=list(map(str, pd.Series(clusters).unique())), filled=True, rounded=True, special_characters=True)  
graph = graphviz.Source(dot_data) 
graph 



In [ ]:
from sklearn.tree import export_text

export_text(clf)

In [ ]:
cluster_to_rules = defaultdict(set)
feature = clf.tree_.feature
threshold = clf.tree_.threshold

for i in range(len(X)):
    X_test = X[i].reshape(1,-1)
    rules = []

    node_indicator = clf.decision_path(X_test)
    leaf_id = clf.apply(X_test)

    sample_id = 0
    # obtain ids of the nodes `sample_id` goes through, i.e., row `sample_id`
    node_index = node_indicator.indices[
        node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
    ]

    for node_id in node_index:
        # continue to the next node if it is a leaf node
        if leaf_id[sample_id] == node_id:
            continue

        # check if value of the split feature for sample 0 is below threshold
        if X_test[sample_id, feature[node_id]] <= threshold[node_id]:
            threshold_sign = "<="
        else:
            threshold_sign = ">"

        f = FEATURE_COLS[feature[node_id]]
        t = threshold[node_id]
        rules.append((f, threshold_sign, t))
            
#         print(
#             "{feature} "
#             "{inequality} {threshold}".format(
#                 node=node_id,
#                 sample=sample_id,
#                 feature=f,
#                 value=X_test[sample_id, feature[node_id]],
#                 inequality=threshold_sign,
#                 threshold=threshold[node_id],
#             )
#         )
        
    cluster_to_rules[Y[i]].add(tuple(rules))

In [ ]:
cluster_to_text_rules = dict()
for cluster, rule in cluster_to_rules.items():
    cluster_to_text_rules[cluster] = 

In [ ]:
def compress(grp):
    if len(grp) == 1:
        return grp

    grp = grp.sort_values('threshold')
    if grp['sign'].iloc[0] == '>':
        return grp.iloc[[-1]]
    else:
        return grp.iloc[[0]]

def shorten_rule(rule):
    df = pd.DataFrame(rule, columns=['feature', 'sign', 'threshold'])
    return list(df.groupby(['feature', 'sign']).apply(compress).itertuples(index=False, name=None))

In [ ]:
# TODO: Can compress same signs
cluster_to_text_rule = dict()

for cluster_id, rules in cluster_to_rules.items():
    decision_string = ''
    for rule in rules:
        rule = shorten_rule(rule)
        if len(decision_string) > 0:
            decision_string += " OR "
        rule_string = '('
        for decision in rule:
            if len(rule_string) > 1:
                rule_string += " AND "
            rule_string += ' '.join(map(str,decision))
        decision_string += rule_string + ')'
    cluster_to_text_rule[cluster_id] = decision_string
with open('rule_dict.pkl', 'wb') as f:
    pickle.dump(cluster_to_text_rule, f)

In [ ]:
cluster_to_text_rule